## Метод простых итераций


Вариант 17

In [1]:
import numpy as np

In [2]:
N = 4
A = np.array([-19, 2, -1, -8, 2, 14, 0, -4, 6, -5, -20, -6, -6, 4, -2, 15])
A = np.reshape(A, (N, N))
b = np.array([38, 20, 52, 43])
print("Матрица системы: \n" + str(A) + "\nПравая часть: \n" + str(b))

Матрица системы: 
[[-19   2  -1  -8]
 [  2  14   0  -4]
 [  6  -5 -20  -6]
 [ -6   4  -2  15]]
Правая часть: 
[38 20 52 43]


Проверим условие диагонального преобладания матрицы по строкам

In [285]:
check = np.zeros(N)
count_z = 0

for i in range(N):
    row_sum = 0
    for j in range(N):
        if i != j:
            row_sum += abs(A[i, j])
    if abs(A[i, i]) > row_sum:
        check[i] = 1
        
count_z       
for c in check:
    if c == 0:
        count_z += 1
if count_z > 0:
    print("Матрица системы не является диагонально преобладающей по строкам")
else: 
    print("Матрица системы - диагонально преобладающая по строкам")


Матрица системы - диагонально преобладающая по строкам


Это означает, что можно использовать метод Якоби.

Преобразования метода Якоби можно представить в матричном виде следующим образом:

1) Исходная система: 
$$A x = b$$

2) Выделим матрицу D, состоящую из элементов главной диагонали матрицы A (все остальные элементы - нули):
$$ A = D + (A - D) $$

3) Представим x в виде:
$$x = D^{-1} (D - A) x + D^{-1} b\;.$$

4) Тогда метод простых итераций примет вид:
$$x_{n + 1} = B x_{n} + c\;,$$

где

$$B = D^{-1} (D - A) \qquad \text{и} \qquad c = D^{-1} b $$

In [286]:
def norm_2(A): # считает 2-норму для матрицы
    sum_sq = 0
    for x in A:
        for y in x:
            sum_sq += y ** 2
    return np.sqrt(sum_sq)

def norm_2_vec(x): # счиатет 2-норму для вектора
    sum_sq = 0
    for y in x:
        sum_sq += y ** 2
    return np.sqrt(sum_sq)

def Jacobi_iter(A, b, max_iter, eps): 
    diag_1 = np.diag(A) # выделяем диагональные элементы
    B = -A.copy()
    np.fill_diagonal(B, 0) # D - A
    D = np.diag(diag_1) 
    invD = np.diag(1./diag_1) 
    BB = invD @ B # матрица B
    c = invD @ b # коэффициенты c
    norm_BB = norm_2(BB) #нужно для условия останова
    coef = norm_BB / (1-norm_BB)
    
    x = c # берём x(0) = c
    for i in range(max_iter): 
        x_prev = x
        x = BB @ x + c
        if coef * norm_2_vec(x - x_prev) < eps:
            break
    
    return x, i #возвратим также количество итераций 

In [287]:
eps = 1e-8
x, i = Jacobi_iter(A, b, 1000, eps)
print("Решение системы x = " + str(x) + " достигнуто за " +str(i) + " итераций с точностью " + str(eps))

#Самопроверка
np.testing.assert_almost_equal(A @ x - b, 0)

Решение системы x = [-2.  2. -4.  1.] достигнуто за 38 итераций с точностью 1e-08


Построим априорную оценку числа итераций для обоих методов и сравним с числом итераций, которое получилось у нас.

In [290]:

B = -A.copy()
np.fill_diagonal(B, 0) # D - A
D = np.diag(diag_1) 
invD = np.diag(1./diag_1) 
BB = invD @ B # матрица B
c = invD @ b # коэффициенты c

expr = (np.log(eps) - np.log(norm_2_vec(c)) + np.log(1-norm_2(BB)))/np.log(norm_2(BB))
expr

182.27952601426156

Получилось очень сильно завышенное число итераций.

## Метод Зейделя


Запишем формулу для итерации методом Зейделя в форме: 

$$ Dx_{n+1} + Lx_{n+1} + Ux_{n} $$

где L - нижняя треугольная матрица с нулями на главной диагонали, полученная из матрицы системы, U - верхняя треугольная с нулями на главной диагонали, D - матрица с главной диагональю исходной матрицы и нулями в остальных местах.

Эту формулу можно переписать в следующем виде:

$$ x_{n+1} = -(D + L)^{-1}Ux_{n} + (D + L)^{-1}b $$

и использовать в качестве формулы для метода простых итераций

In [269]:
def Seidel_iter(A, b, max_iter, eps):
    
    B = A.copy()
    diag_1 = np.diag(B)
    D = np.diag(diag_1) #Матрица с главной диагональю исходной матрицы и нулями в остальных местах
    L = np.tril(B, -1) #Нижняя треугольная с нулями на главной диагонали
    U = np.triu(B, 1) #Верхняя треугольная с нулями на главной диагонали
    
    DpL_inv = np.linalg.inv(D + L)
    BB = DpL_inv @ U
    c = DpL_inv @ b
    x = b
    
    U_norm = norm_2(U)
    BB_norm = norm_2(BB)
    coef = U_norm / (1-BB_norm)
    
    for i in range(max_iter):
        x_prev = x
        x = -BB @ x + c
        if coef * norm_2_vec(x - x_prev) < eps:
            break
    return x, BB, i #return BB only to check conditions of convergence later on 

In [273]:
eps = 1e-8
x2, BB, i = Seidel_iter(A, b, 1000, eps)

print("Решение системы x = " + str(x) + " достигнуто за " +str(i) + " итераций с точностью " + str(eps))

Решение системы x = [-2.  2. -4.  1.] достигнуто за 31 итераций с точностью 1e-08


Для решения системы методом Зейделя потребовалось на 7 итераций меньше

In [291]:
B = A.copy()
diag_1 = np.diag(B)
D = np.diag(diag_1) #Матрица с главной диагональю исходной матрицы и нулями в остальных местах
L = np.tril(B, -1) #Нижняя треугольная с нулями на главной диагонали
U = np.triu(B, 1) #Верхняя треугольная с нулями на главной диагонали
    
DpL_inv = np.linalg.inv(D + L)
BB = DpL_inv @ U
c = DpL_inv @ b
x = b

expr = (np.log(eps) - np.log(norm_2_vec(c)) + np.log(1-norm_2(BB)))/np.log(norm_2(BB))
expr

115.7090285954391

Снова получилось очень сильно завышенное число итераций.